In [1]:
import requests
import pandas as pd
import json
import datetime as dt
import time

from sqlalchemy import create_engine, text

<h3> POSTGRES Set Up </h3>

In [2]:
with open ('postgre_aut.txt', 'r') as f:
    aut = f.read()[:-1]

In [3]:
engine = create_engine(aut)

with engine.connect() as conn:
    
    data = conn.execute(text('SELECT * FROM fx_price_collected;'))
    

df_bank = pd.DataFrame(data)
if not df_bank.empty:
    df_bank = df_bank.set_index(['coin', 'date'])

df_bank.head(2)

,,code,codein,name,high,low,varBid,pctChange,bid,ask
coin,date,,,,,,,,,
USDBRL,2024-11-04 15:16:53,USD,BRL,Dólar Americano/Real Brasileiro,5.8697,5.7561,-0.0894,-1.52,5.7782,5.7792
EURBRL,2024-11-04 15:16:38,EUR,BRL,Euro/Real Brasileiro,6.3579,6.203,0.0859,1.38,6.2849,6.2929


<h2> API that gives the FX prices </h2>

In [6]:
url = 'https://economia.awesomeapi.com.br/json/last/USD-BRL,EUR-BRL'    
def api_call():

    r = requests.get(url) # calls

    df = pd.DataFrame(json.loads(r.text)).T

    #transform.
    df.index = pd.MultiIndex.from_arrays([df.index, df.create_date.values], names = ['coin', 'date'])
    df = df.drop(columns = ['timestamp', 'create_date'])
    
    return df

<h3> POST to register a receiver e-mail  </h3>

In [15]:
### register an e-mail for subs

webhook_ad = 'http://127.0.0.1:5000/' # first, set up the webhook endpoint.
headers={'Content-Type': 'application/json'}

# set the mail_address as a list of e-mails as such: ['alexa@gmail.com']
mail_post = {'type': 'inclusion', 
             'name': ['July'],
             'mail_address': ['joelyjulyjuju@gmail.com']}


include_mail = requests.post(webhook_ad, headers=headers, data = json.dumps(mail_post))

assert include_mail.status_code == 200

print(f"API response: {include_mail.text}")


API response: "e-mail already registered"



<h3> API call and webhook triger </h3>

In [18]:
new_data = api_call()
mask = new_data.index.isin(df_bank.index) # checks if diferent pair coin:time arrived;

if new_data[~mask].empty:
    print('No new data on the LAST API CALL!')

else:
    print("New data found! Here its")
    display(new_data[~mask])

    # append new data -> save on postgres
    new_data[~mask].reset_index().to_sql('fx_price_collected', engine, if_exists='append', index = False)

    # trigering the webhook;  
    json_data = new_data[~mask].reset_index().to_json()
    json_data = json.loads(json_data)

    # sending a post
    post_msg = {"type": "alert", 
                "data" : json_data}
    
    r = requests.post(webhook_ad, json.dumps(post_msg), headers=headers)
    
    print(f'\nWEBHOOK response: {r.text}')


New data found! Here its


,,code,codein,name,high,low,varBid,pctChange,bid,ask
coin,date,,,,,,,,,
USDBRL,2024-11-04 17:58:59,USD,BRL,Dólar Americano/Real Brasileiro,5.8693,5.7553,-0.0005,0,5.7916,5.7926
EURBRL,2024-11-04 23:48:07,EUR,BRL,Euro/Real Brasileiro,6.2997,6.2997,0,0,6.2957,6.3037



WEBHOOK response: {
  "message": "New coin prices received!"
}

